## Importing libraries

In [2]:
from collections import Counter

## Obtaining the results on the validation set

Sending the model to CPU

In [3]:
# model.to('cpu')
import torch
import pickle
from transformers import AutoTokenizer
from transformers import BertForQuestionAnswering

filepath = '../models/m-bert.pth' # saving weights
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")
model.load_state_dict(torch.load(filepath))
model.eval() # model summary

with open("../data/val_answers.pkl","rb") as file:
    val_answers = pickle.load(file)

with open("../data/val_contexts.pkl","rb") as file:
    val_contexts = pickle.load(file)

with open("../data/val_questions.pkl","rb") as file:
    val_questions = pickle.load(file)

/home/orian/PycharmProjects/ANLP/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Getting the model answers to evaluate

In [4]:
eval_answers = []

for i in range(len(val_questions)):
  question, text = val_questions[i], val_contexts[i]

  input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"

  input_ids = tokenizer.encode(input_text)
  token_type_ids = [0 if i <= input_ids.index(102) else 1
      for i in range(len(input_ids))]

  model_response = model(torch.tensor([input_ids]), \
      token_type_ids=torch.tensor([token_type_ids]))
  
  start_scores = model_response['start_logits']
  end_scores = model_response['end_logits']

  all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
  ans = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])

  eval_answers.append(ans)

Computing Precision, Recall and F1-Score

In [5]:
import collections

num_c = []
num_p = []
num_g = []

for a in range(len(eval_answers)):

  common = collections.Counter(eval_answers[a].split()) & collections.Counter(eval_answers[a].split()) # tokens shared between gold and predicted answers
  num_common = sum(common.values())

  num_pred = len(str(eval_answers[a]).split()) # the number of predicted tokens

  num_gold = len(str(val_answers[a]).split()) # the number of gold tokens

  num_c.append(num_common)
  num_p.append(num_pred)
  num_g.append(num_gold)

precision = 1.0 * sum(num_c) / sum(num_p) # the num of tokens shared between gold and predicted answers / the num of predicted tokens
recall = 1.0 * sum(num_c) / sum(num_g) # the num of tokens shared between gold and predicted answers / the num of gold tokens
f1_score= (2 * precision * recall) / (precision + recall)

In [6]:
f1_score

0.7358153007326632